Trains and  tests on parts of  NA12878 data for Del_st, Del_end and No_sv, ground  truth Mills_nanosv

## Load dependences and setting output configuration

In [1]:
import numpy as np
from keras.utils.np_utils import to_categorical
import keras
%pylab inline
%load_ext autoreload
%autoreload 2

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Populating the interactive namespace from numpy and matplotlib


## Load data from npy files

In [2]:
import pickle
dico = pickle.load(open("/home/ubuntu/GTCG_labels/labels.pickle", "rb"))
data_labels = dataset = dico["Mills2011_nanosv"]
window_size = 200

In [3]:
dico = []

In [4]:
replications = 500
factor_nosv = 1 # factor_nosv times less Nosv than del-st

In [5]:
def eliminate_noise_chanels(X):
    return np.delete(X, [4-1,5-1,6-1,7-1,8-1,9-1,10-1,12-1,13-1,15-1,16-1, 17-1,18-1,19-1,23-1,24-1,26-1,27-1], axis=1)
    

In [6]:
def eliminate_distance_chanels(X):
    return np.delete(X, [3,4,7,8], axis=1)

In [7]:
datapath = '/home/ubuntu/GTCG_data' 

germline_1_datapath = datapath + '/'
#ger_1_labels_datapath = datapath + '/LabelData/unzipped/'

G1_data_SV = np.empty([0, 5, window_size])
G1_labels_SV =[]
G1_data_noSV = np.empty([0, 5, window_size])
G1_labels_noSV =[]
total_train_sv_windows = 0
for i in range(1,23,1):
        G1_data_new =np.load(germline_1_datapath + str(i)  +'_channel_maker_real_germline.npy')
        #G1_labels_new = np.load(ger_1_labels_datapath  +  str(i) +'_label_ci_full_overlap.npy/' +str(i) +'_label_ci_full_overlap.npy') 
        G1_labels_new = np.array(data_labels[str(i)])
        print(G1_labels_new[0])
        G1_data_new = G1_data_new[np.where(G1_labels_new != 'UK')]
        G1_data_new = eliminate_noise_chanels(G1_data_new)
        G1_data_new = eliminate_distance_chanels(G1_data_new)
        G1_labels_new = G1_labels_new[np.where(G1_labels_new != 'UK')]
        G1_data_SV_new  = G1_data_new[np.where(G1_labels_new != 'noSV')]
        G1_labels_SV_new  = G1_labels_new[np.where(G1_labels_new != 'noSV')]
        n_windows_sv = G1_data_SV_new.shape[0]
        G1_data_noSV_new  = G1_data_new[np.where(G1_labels_new == 'noSV')]
        G1_labels_noSV_new  = G1_labels_new[np.where(G1_labels_new == 'noSV')]
        G1_data_noSV_new =  G1_data_noSV_new[0:int(replications*n_windows_sv/2/factor_nosv)]# 4 times less than del_st to boost recall
        G1_labels_noSV_new =  G1_labels_noSV_new[0:int(replications*n_windows_sv/2/factor_nosv)] # also
        print(str(i) + ':' + str(G1_data_SV_new.shape))
        #print(G1_labels_SV_new[0])
        G1_data_SV = np.concatenate((G1_data_SV, G1_data_SV_new))
        G1_labels_SV = np.concatenate((G1_labels_SV, G1_labels_SV_new))
        G1_data_noSV = np.concatenate((G1_data_noSV, G1_data_noSV_new))
        G1_labels_noSV = np.concatenate((G1_labels_noSV, G1_labels_noSV_new))
        if i < 16: #for training until chromozome 16
            total_train_sv_windows += G1_data_SV_new.shape[0]

noSV
1:(222, 5, 200)
noSV
2:(205, 5, 200)
noSV
3:(208, 5, 200)
noSV
4:(228, 5, 200)
noSV
5:(214, 5, 200)
noSV
6:(162, 5, 200)
noSV
7:(163, 5, 200)
noSV
8:(135, 5, 200)
noSV
9:(127, 5, 200)
noSV
10:(107, 5, 200)
noSV
11:(119, 5, 200)
noSV
12:(114, 5, 200)
noSV
13:(106, 5, 200)
noSV
14:(66, 5, 200)
noSV
15:(61, 5, 200)
noSV
16:(47, 5, 200)
noSV
17:(48, 5, 200)
noSV
18:(77, 5, 200)
noSV
19:(44, 5, 200)
noSV
20:(58, 5, 200)
noSV
21:(28, 5, 200)
noSV
22:(21, 5, 200)


### Find  the indexes that split the categories

In [8]:
def find_splitting_indices(labels):
    for i in range (0, labels.shape[0] - 1):
        if (labels[i] != labels[i+1]):
            print(i , ': ', labels[i] , labels[i+1] )        

In [9]:
#find_splitting_indices(G1_labels)

In [10]:
#G1_labels_SV = ['DEL'] * G1_labels_SV.shape[0]

In [11]:
sv=G1_data_SV.shape[0]

In [12]:
sv

2560

In [13]:
int(sv/2)

1280

In [14]:
nosv = G1_data_noSV.shape[0]
nosv

640000

In [15]:
#G1_data_noSV  = G1_data_noSV[0:2*sv] 

In [16]:
#G1_labels_noSV  = G1_labels_noSV[0:2*sv] 

In [17]:
SV_train =  int(0.8 * total_train_sv_windows)
noSV_train = int(replications * SV_train / 2/factor_nosv)
SV_val = total_train_sv_windows - SV_train
noSV_val = int(replications * SV_val / 2/factor_nosv)

In [18]:
def replicate(data, start, end, howmanytimes): 
    X = data[start:end]
    for i in range(0,howmanytimes):
        X = np.concatenate((X, data[start:end]))
    return X    
    

In [19]:
X_train = replicate(G1_data_SV, 0, SV_train, replications)
y_train = replicate(G1_labels_SV, 0, SV_train,replications)
X_val = replicate(G1_data_SV, SV_train, SV_train+SV_val, replications)
y_val = replicate(G1_labels_SV, SV_train, SV_train+SV_val, replications)
X_test = replicate(G1_data_SV, SV_train+SV_val, len(G1_data_SV), replications)
y_test = replicate(G1_labels_SV, SV_train+SV_val, len(G1_labels_SV), replications)

In [20]:
G1_data_SV = []
G1_labels_SV = []

In [21]:
X_train.shape[1]

5

lrm 

In [22]:
#np.save(germline_1_datapath  + "X_train_SV.npy", X_train)
#np.save(germline_1_datapath  + "y_train_SV.npy", y_train)
#np.save(germline_1_datapath  + "X_val_SV.npy", X_val)
#np.save(germline_1_datapath  + "y_val_SV.npy", y_val)
#np.save(germline_1_datapath  + "X_test_SV.npy", X_test)
#np.save(germline_1_datapath  + "y_test_SV.npy", y_test)
#np.save(germline_1_datapath  + "NoSVall", G1_data_noSV)
#np.save(germline_1_datapath  + "NoSV_train", G1_data_noSV[0:noSV_train ])
#np.save(germline_1_datapath  + "NoSV_val", G1_data_noSV[noSV_train:noSV_train +noSV_val ])
#np.save(germline_1_datapath  + "NoSV_test", G1_data_noSV[noSV_train +noSV_val:])
#np.save(germline_1_datapath  + "y_train_NoSV", G1_labels_noSV[0:noSV_train ])
#np.save(germline_1_datapath  + "y_val_NoSV", G1_labels_noSV[noSV_train:noSV_train +noSV_val])
#np.save(germline_1_datapath  + "y_test_NoSV", G1_labels_noSV[noSV_train +noSV_val:])

In [23]:
X_train = np.concatenate((X_train, G1_data_noSV[0:noSV_train ]))
X_val = np.concatenate((X_val, G1_data_noSV[noSV_train:noSV_train +noSV_val ]))
X_test = np.concatenate((X_test, G1_data_noSV[noSV_train +noSV_val:]))

In [24]:
y_train = np.concatenate((y_train, G1_labels_noSV[0:noSV_train ]))
y_val = np.concatenate((y_val, G1_labels_noSV[noSV_train:noSV_train +noSV_val ]))
y_test = np.concatenate((y_test, G1_labels_noSV[noSV_train +noSV_val:]))

In [25]:
G1_data_noSV = []
G1_labels_noSV = []

### Number of training, validation and test windows

In [26]:
X_train.shape

(1343539, 5, 200)

In [27]:
X_val.shape

(336448, 5, 200)

In [28]:
X_test.shape

(242573, 5, 200)

In [29]:
y_test.shape

(242573,)

In [30]:
number_channels = X_train.shape[1]
number_channels

5

## Normalize data using the reads coverage as parameter and remove uninteresting channels for deletions

In [31]:
coverage = np.median(X_train[:, 0, :]) #the median of the coverage in the normal cells
coverage


36.0

In [32]:
def normalize(X):       
    for i in range (0, len(X)):                              
        for j in range(0,number_channels):                                                                                            
            X[i][j] = np.divide(X[i][j], coverage)*100 
normalize(X_train)
normalize(X_val)
normalize(X_test)

In [33]:
#X_train = eliminate_noise_chanels(X_train)
#X_val = eliminate_noise_chanels(X_val)
#X_test = eliminate_noise_chanels(X_test)

In [34]:
#X_train = eliminate_distance_chanels(X_train)
#X_val = eliminate_distance_chanels(X_val)
#X_test = eliminate_distance_chanels(X_test)

In [35]:
X_train.shape

(1343539, 5, 200)

In [36]:
number_channels = X_train.shape[1]
number_channels

5

## Differentiate the time series for a zero mean

In [37]:
def differentiate(X):
    for i in range(0, X.shape[0]):        
        for j in range(0, X.shape[1]):
            for k in range(0, X.shape[2]-1):
                X[i][j][k] = X[i][j][k+1] - X[i][j][k]
    return X[:,:,:-1]        

In [38]:
#X_train = differentiate(X_train)

In [39]:
#X_val = differentiate(X_val)
#X_test = differentiate(X_test)

## Plots several  windows

In [40]:
n_windows = 100
label = ["None"] * number_channels

In [41]:
label[0] = "germline:coverage"
label[1] = "germline:#left clipped reads"
label[2] = "germline:#right clipped reads"
label[3] = "germline:#split reads right split"
label[4] = "germline:#split reads left split"

In [42]:
from numpy import *
import math
import matplotlib.pyplot as plt

colors =['b','g', 'm', 'gray', 'r','c', 'darkgreen' ,'y', 'k', 'orange', 'gray']

def plot_channels(start_window, n_windows):
    for i in range (start_window, start_window + n_windows, 1):
        #if y_test[i] == labels[predicted[i]]:
            #continue
        #print(y_train[i], 'id:', i)
        print('id:', i)
        print('Label:', y_test[i])#, '. DeepSV:', labels[predicted[i]])
        for j in range (0, number_channels):   
            shift = 0
            start =0
            if j in [0,11]:
                shift = -100
            if j > 8:
                start = -60
            Z = [start + shift + 300 + x - 5*j*4 for x in X_test[i][j]]  
            plt.ylim([150,450])
            if np.max(Z) > 1000:
                plt.ylim([-200,1000])
                Z = [start + shift + 300 + x - 5*j*4 - 400 for x in X_test[i][j]]  
            if j>8:
                plt.plot(Z,  label = label[j], linestyle=':', linewidth=1.5, color = colors[j % 9])
            else:
                plt.plot(Z,  label = label[j], color = colors[j])
        
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., prop={'size': 10})    
        plt.show()

In [43]:
start_window = 0
n_windows = 57
#plot_channels(start_window, n_windows)

In [44]:
n_windows = 50
start_window = 500
#plot_channels(start_window, n_windows)

   
## Transposes every window in X, to comply to McFly format

In [45]:
def transposeDataset(X):
    image = []
    for i in range (0, len(X -1)):
        tr = X[i].transpose()
        image.append(tr)
    return np.array(image)

image1 = transposeDataset(X_train)
image2 = transposeDataset(X_val)
image3 = transposeDataset(X_test)
X_train = image1
X_val = image2
X_test = image3

n_channels = len(X_train[0][0])
print(len(X_test[0][0]))

5


## Prepare the labels in McFly format

In [46]:
mapclasses = {'DEL_start': 1, 'DEL_end': 0,  'noSV': 2}
print(mapclasses)

{'DEL_start': 1, 'DEL_end': 0, 'noSV': 2}


In [47]:
y_train = np.array([mapclasses[c] for c in y_train], dtype='int')
y_val = np.array([mapclasses[c] for c in y_val], dtype='int')
y_test = np.array([mapclasses[c] for c in y_test], dtype='int')
y_train_binary = to_categorical(y_train)
y_val_binary = to_categorical(y_val)
y_test_binary = to_categorical(y_test)

In [48]:
y_test_binary

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [49]:
y_val_binary

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [50]:
X_train.shape

(1343539, 200, 5)

##  Generate and train neural networks 

In [51]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D
from keras.optimizers import Adam
import sys
import os
import numpy as np
sys.path.insert(0, os.path.abspath('..'))
from mcfly import modelgen, find_architecture

In [52]:
np.random.seed = 321
num_classes = y_train_binary.shape[1]

%%time
models = modelgen.generate_models(X_train.shape,
                                  num_classes,
                                  number_of_models = 1,
                                  model_type = 'CNN',
                                  cnn_min_layers=1,
                                  cnn_max_layers=1,
                                  cnn_min_filters = 3, 
                                  cnn_max_filters = 3, 
                                  cnn_min_fc_nodes=2, 
                                  cnn_max_fc_nodes=2,
                                  low_lr=1.73, high_lr=1.73,
                                  kernel_size = 11)

In [53]:
%%time
models = modelgen.generate_models(X_train.shape,                                                                   
                                  num_classes,                                  
                                  number_of_models = 15,
                                  model_type = 'CNN',
                                  cnn_min_layers=2,
                                  cnn_max_layers=2,
                                  cnn_min_filters =6, 
                                  cnn_max_filters =16, 
                                  cnn_min_fc_nodes=4, 
                                  cnn_max_fc_nodes = 10,
                                  low_lr=1.9, high_lr=2.0, # 1.97781612,
                                  kernel_size = 5)

CPU times: user 8.44 s, sys: 128 ms, total: 8.57 s
Wall time: 8.67 s


In [54]:
%%time
i=0
for model, params, model_types in models:
    print('model ' + str(i))
    i=i+1
    print(params)
    model.summary()

model 0
{'learning_rate': 0.010968974486113869, 'regularization_rate': 0.0223041065539198, 'filters': array([ 9, 13]), 'fc_hidden_nodes': 5, 'kernel_size': 5}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 200, 5)            20        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 200, 9)            234       
_________________________________________________________________
batch_normalization_2 (Batch (None, 200, 9)            36        
_________________________________________________________________
activation_1 (Activation)    (None, 200, 9)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 200, 13)           598       
_________________________________________________________________
batch_normalization_3 (Batch (None, 200, 13)     

In [55]:
train_set_size = X_train.shape[0]
train_set_size

1343539

In [56]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16381893074357336544
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11791269888
locality {
  bus_id: 1
}
incarnation: 16544931794256808006
physical_device_desc: "device: 0, name: Tesla P100-PCIE-12GB, pci bus id: 0000:01:01.0, compute capability: 6.0"
]


In [57]:
%%time
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                                 X_val, y_val_binary,
                                                                                 models,nr_epochs=1, 
                                                                                 subset_size=train_set_size,
                                                                                 verbose=True)

Training model 0 CNN
Train on 1343539 samples, validate on 336448 samples
Epoch 1/1
1343539/1343539 [==============================] - 1634s 1ms/step - loss: 0.9628 - acc: 0.4576 - val_loss: 9.7582 - val_acc: 0.3484
Training model 1 CNN
Train on 1343539 samples, validate on 336448 samples
Epoch 1/1
1343539/1343539 [==============================] - 1653s 1ms/step - loss: 0.2091 - acc: 0.9626 - val_loss: 0.3339 - val_acc: 0.9349
Training model 2 CNN
Train on 1343539 samples, validate on 336448 samples
Epoch 1/1
1343539/1343539 [==============================] - 1642s 1ms/step - loss: 0.1171 - acc: 0.9779 - val_loss: 0.4188 - val_acc: 0.9146
Training model 3 CNN
Train on 1343539 samples, validate on 336448 samples
Epoch 1/1
1343539/1343539 [==============================] - 1659s 1ms/step - loss: 0.1122 - acc: 0.9785 - val_loss: 6.3037 - val_acc: 0.3264
Training model 4 CNN
Train on 1343539 samples, validate on 336448 samples
Epoch 1/1
1343539/1343539 [==============================] - 1

## Save the best model

In [58]:
best_model_index = np.argmax(val_accuracies)
best_model, best_params, best_model_types = models[best_model_index]
print(best_model_index, best_model_types, best_params)

8 CNN {'learning_rate': 0.01143887106914512, 'regularization_rate': 0.014119965244790249, 'filters': array([16, 10]), 'fc_hidden_nodes': 4, 'kernel_size': 5}


In [59]:
from keras.models import load_model

model_path = os.path.join('/home/ubuntu/CNN/notebooks/', 'Gtcg_07_11_500xoversample_NoDiff_9527')

best_model.save(model_path)

## Reload the trained model

In [60]:
import os
import keras
from keras.models import load_model
model_path = os.path.join('/home/ubuntu/CNN/notebooks/',  'Gtcg_07_11_1500xoversample_NoDiff_')
model_reloaded = load_model(model_path)

In [61]:
nr_epochs = 1
datasize = train_set_size # Change in `X_train.shape[0]` if training complete data set
#history = model_reloaded.fit(X_train[:datasize,:,:], y_train_binary[:datasize,:],
 #             epochs=nr_epochs, validation_data=(X_val, y_val_binary))

## Inspect model predictions on test data

In [62]:

model = model_reloaded 
datasize = X_test.shape[0]
probs = model.predict_proba(X_test[:datasize,:,:],batch_size=1)

## Columns are predicted labels by DeepSV, rows are nanopore labels

In [63]:
mapclasses = {'DEL_end': 0, 'DEL_start': 1, 'noSV': 2}
dict=mapclasses
dict_sorted = sorted(dict.items(), key=lambda x: x[1])
dict_sorted

[('DEL_end', 0), ('DEL_start', 1), ('noSV', 2)]

In [64]:
labels = [i[0] for i in dict_sorted]

In [65]:
import pandas as pd
predicted = probs.argmax(axis=1)
y_index = y_test_binary.argmax(axis=1)
confusion_matrix = pd.crosstab(pd.Series(y_index), pd.Series(predicted))
confusion_matrix.index = [labels[i] for i in confusion_matrix.index]
confusion_matrix.columns = [labels[i] for i in confusion_matrix.columns]
confusion_matrix.reindex(columns=[l for l in labels], fill_value=0)
confusion_matrix

,DEL_end,DEL_start,noSV
DEL_end,68136,1503,6012
DEL_start,2505,78657,5010
noSV,640,742,79368


## Test best current model on testset

### Overall accuracy on the test set:

In [66]:
%%time
score_test = model.evaluate(X_test, y_test_binary, verbose=False)
print('Test loss and accuracy of best model: ' + str(score_test))

Test loss and accuracy of best model: [0.3572650673766878, 0.9323420166300453]
CPU times: user 41.3 s, sys: 1.5 s, total: 42.8 s
Wall time: 43.4 s


In [67]:
5

5